In [1]:
# Cell 1 - (optional) install runtime deps (uncomment if needed)
%pip install MetaTrader5 stable-baselines3 gymnasium numpy pandas matplotlib joblib
#!{sys.executable} -m pip install MetaTrader5 stable-baselines3 gymnasium numpy pandas matplotlib joblib


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Cell 2 - Imports
import os, glob, time, traceback, json
from datetime import datetime
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import MetaTrader5 as mt5
import gymnasium as gym
import joblib
import matplotlib.pyplot as plt

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize


In [3]:
# Cell 3 - Config (edit paths & settings here)
DRY_RUN = True            # SAFE DEFAULT: True => no real trades (set False only after testing)
WINDOW = 50               # must match training window
BUFFER = 60
COUNT = WINDOW + BUFFER

TIMEFRAME = "M15"         # "M1","M5","M15","M30","H1","H4","D1"
TF_MAP = {
    "M1": mt5.TIMEFRAME_M1, "M5": mt5.TIMEFRAME_M5, "M15": mt5.TIMEFRAME_M15,
    "M30": mt5.TIMEFRAME_M30, "H1": mt5.TIMEFRAME_H1, "H4": mt5.TIMEFRAME_H4,
    "D1": mt5.TIMEFRAME_D1
}
TF_MT5 = TF_MAP.get(TIMEFRAME.upper(), mt5.TIMEFRAME_M15)

DATA_DIR = os.path.join("data", "multiasset")
MODEL_DIR = os.path.join("models", "multiasset")
EMBED_FILE = os.path.join(MODEL_DIR, "asset_embeddings.npy")
ASSET_MAP_FILE = os.path.join(DATA_DIR, "asset_to_idx.csv")
MODEL_FILE = os.path.join(MODEL_DIR, "ppo_multiasset.zip")
VEC_FILE = os.path.join(MODEL_DIR, "vec_normalize.pkl")

LOG_DIR = os.path.join(MODEL_DIR, "live_logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, "live_trade_logs.csv")

DEFAULT_LOT = 0.1
MAX_POS_PER_SYMBOL = 2
TP_MULT = 3
DEFAULT_SL_PIPS_FALLBACK = 20
TRAIL_PIPS_DEFAULT = 5

# Symbols list you trained on / want to trade (use broker exact names if needed)
SYMBOLS = [
    "Volatility 10 Index","Volatility 25 Index","Volatility 50 Index","Volatility 75 Index","Volatility 100 Index",
    "Volatility 10 (1s) Index","Volatility 25 (1s) Index","Volatility 50 (1s) Index","Volatility 75 (1s) Index","Volatility 100 (1s) Index",
    "Volatility 10 (10s) Index","Volatility 25 (10s) Index","Volatility 50 (10s) Index","Volatility 75 (10s) Index","Volatility 100 (10s) Index",
    "Jump 10 Index","Jump 25 Index","Jump 50 Index","Jump 75 Index","Jump 100 Index",
    "Step Index 25","Step Index 50","Step Index 75","Step Index 100","EURUSD"
]

# Map safe names to broker MT5 names if broker uses different naming.
MANUAL_SYMBOL_MAP: Dict[str, str] = {
    # e.g. "Volatility_75_Index": "Deriv-Vol75",
}


In [4]:
# Cell 4 - MT5 connect / helper utilities
def mt5_connect() -> bool:
    try:
        if not mt5.initialize():
            # try again
            ok = mt5.initialize()
            if not ok:
                print("MT5 initialize failed:", mt5.last_error())
                return False
        print("MT5 connected, version:", mt5.version())
        return True
    except Exception as e:
        print("MT5 connection error:", e)
        return False

def mt5_shutdown():
    try:
        mt5.shutdown()
    except Exception:
        pass

def make_safe_name(sym: str) -> str:
    return sym.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace(".", "_")

def get_mt5_symbol_from_safe(safe_name: str) -> str:
    # default inverse transformation
    return MANUAL_SYMBOL_MAP.get(safe_name, safe_name.replace("_", " "))


In [5]:
# Cell 5 - Load model and preprocessors
# MODEL_FILE, VEC_FILE, EMBED_FILE, DATA_DIR should be configured in Cell 3

# Load PPO model
if not os.path.exists(MODEL_FILE):
    raise FileNotFoundError("Model file missing: " + MODEL_FILE)
model = PPO.load(MODEL_FILE)
print("Loaded PPO model:", MODEL_FILE)

# Load embeddings (handle dict or ndarray)
embeddings: Dict[str, np.ndarray] = {}
EMBED_DIM = 8
if os.path.exists(EMBED_FILE):
    try:
        emb_raw = np.load(EMBED_FILE, allow_pickle=True)
        if isinstance(emb_raw, np.ndarray) and emb_raw.dtype == object:
            # possibly saved dict
            try:
                obj = emb_raw.item()
                if isinstance(obj, dict):
                    embeddings = {k: np.array(v, dtype=np.float32) for k, v in obj.items()}
            except Exception:
                pass
        if len(embeddings) == 0:
            arr = np.array(emb_raw)
            if arr.ndim == 2:
                # try mapping via asset_to_idx
                if os.path.exists(ASSET_MAP_FILE):
                    try:
                        am = pd.read_csv(ASSET_MAP_FILE, index_col=0, header=None).iloc[:,0].to_dict()
                        for safe, idx in am.items():
                            idx = int(idx)
                            if idx < arr.shape[0]:
                                embeddings[safe] = np.array(arr[idx], dtype=np.float32)
                    except Exception:
                        pass
                if len(embeddings) == 0:
                    # fallback map by CSV order
                    csvs = sorted(glob.glob(os.path.join(DATA_DIR, "*_normalized.csv")))
                    safe_list = [os.path.basename(p).replace("_normalized.csv","") for p in csvs]
                    if len(safe_list) == arr.shape[0]:
                        for i, s in enumerate(safe_list):
                            embeddings[s] = np.array(arr[i], dtype=np.float32)
    except Exception as e:
        print("Failed to load embeddings:", e)
if len(embeddings) > 0:
    EMBED_DIM = next(iter(embeddings.values())).shape[0]
print("Embeddings loaded:", len(embeddings), "embed_dim:", EMBED_DIM)

# Safe load VecNormalize using dummy env
vecnorm = None
if os.path.exists(VEC_FILE):
    try:
        class _DummyEnv(gym.Env):
            def __init__(self):
                super().__init__()
                self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(WINDOW, 5 + 1 + EMBED_DIM), dtype=np.float32)
                self.action_space = gym.spaces.Discrete(3)
            def reset(self, seed=None, options=None):
                return np.zeros(self.observation_space.shape, dtype=np.float32), {}
            def step(self, action):
                return np.zeros(self.observation_space.shape, dtype=np.float32), 0.0, True, False, {}
        venv = DummyVecEnv([lambda: _DummyEnv()])
        vecnorm = VecNormalize.load(VEC_FILE, venv)
        vecnorm.training = False
        vecnorm.norm_reward = False
        print("VecNormalize loaded:", VEC_FILE)
    except Exception as e:
        print("VecNormalize load failed (continuing without it):", e)
        vecnorm = None
else:
    print("No VecNormalize file found; continuing without it.")

# Load scalers from CSV files (saved during data prep)
def load_scalers(data_dir=DATA_DIR) -> Dict[str, Dict[str, pd.Series]]:
    scalers = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_scaler.csv"))):
        safe = os.path.basename(p).replace("_scaler.csv","")
        df = pd.read_csv(p, index_col=0)
        # store 'mean' and 'std' Series
        scalers[safe] = {"mean": df["mean"], "std": df["std"]}
    return scalers

# Load prepared normalized datasets (for fallback & indexing)
def load_prepared_datasets(data_dir=DATA_DIR, window=WINDOW) -> Dict[str, pd.DataFrame]:
    datasets = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv"))):
        safe = os.path.basename(p).replace("_normalized.csv","")
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        expected = ['o_pc','h_pc','l_pc','c_pc','v_pc','Close_raw']
        if all(c in df.columns for c in expected):
            tmp = df[expected].dropna()
            if len(tmp) > window:
                datasets[safe] = tmp
        else:
            # try to convert if raw OHLCV present
            if all(c in df.columns for c in ['open','high','low','close','volume']):
                tmp = pd.DataFrame(index=df.index)
                tmp['o_pc'] = df['open'].pct_change()
                tmp['h_pc'] = df['high'].pct_change()
                tmp['l_pc'] = df['low'].pct_change()
                tmp['c_pc'] = df['close'].pct_change()
                tmp['v_pc'] = df['volume'].pct_change()
                tmp['Close_raw'] = df['close']
                tmp = tmp.dropna()
                if len(tmp) > window:
                    datasets[safe] = tmp
    return datasets

scalers = load_scalers()
datasets = load_prepared_datasets()
print("Scalers:", len(scalers), "Datasets:", len(datasets))


Loaded PPO model: models\multiasset\ppo_multiasset.zip
Embeddings loaded: 0 embed_dim: 8
VecNormalize loaded: models\multiasset\vec_normalize.pkl
Scalers: 0 Datasets: 0


In [6]:
# Cell 6 - Live fetch helper
def _fetch_single_symbol_bars(mt5_symbol: str, timeframe: int = TF_MT5, count: int = COUNT) -> Optional[pd.DataFrame]:
    info = mt5.symbol_info(mt5_symbol)
    if info is None:
        return None
    if not info.visible:
        try:
            mt5.symbol_select(mt5_symbol, True)
        except Exception:
            pass
    end_time = datetime.now()
    bars = mt5.copy_rates_from(mt5_symbol, timeframe, end_time, count)
    if bars is None or len(bars) < WINDOW + 2:
        return None
    df = pd.DataFrame(bars)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df.set_index('time')
    if 'tick_volume' in df.columns:
        df = df.rename(columns={'tick_volume':'volume'})
    df = df[['open','high','low','close','volume']].copy()
    df['Close_raw'] = df['close']
    return df
